# SageMaker で提供される Neural Topic Model の学習と推論を行う

#### ノートブックに含まれる内容

- Amazon が提供するアルゴリズムの使いかた
- 中でも，Neural Topic Model の使い方

#### ノートブックで使われている手法の詳細

- アルゴリズム: Neural Topic Model
- データ: スクリプトで自動生成されたサンプルデータ


## セットアップ

In [ ]:
prefix = 'sagemaker/DEMO-ntm-synthetic'
 
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import numpy as np
from generate_example_data import generate_griffiths_data, plot_topic_data
import io
import os
import time
import json
import sys
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import scipy
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker.predictor import csv_serializer, json_deserializer

sess = sagemaker.session.Session()
bucket = sess.default_bucket()

## データの生成

ここでは `generate_example_data.py` を使って，学習用のデータを生成します．文章軍に含まれ全単語のボキャブラリー数，ドキュメント数およびトピック数を指定して，ディリクレ分布にしたがってサンプルデータを生成します．ここで各単語は具体的な文字列ではなく，あらかじめ数字にエンコードされた形であらわされます．SageMaker の Neural Topic Model では，学習用データに単語や文章をそのまま使うことはできません．ドキュメント行 x ボキャブラリー列の行列形式で学習用データを用意する必要があります．各セルの中身は，当該ドキュメントにおける，当該単語の出現回数となります．

In [ ]:
# generate the sample data
vocabulary_size = 25
num_documents = 5000
num_topics = 5

known_alpha, known_beta, documents, topic_mixtures = generate_griffiths_data(
    num_documents=num_documents, num_topics=num_topics, vocabulary_size=vocabulary_size)

# separate the generated data into training and tests subsets
num_documents_training = int(0.8*num_documents)
num_documents_test = num_documents - num_documents_training

documents_training = documents[:num_documents_training]
documents_test = documents[num_documents_training:]

topic_mixtures_training = topic_mixtures[:num_documents_training]
topic_mixtures_test = topic_mixtures[num_documents_training:]

data_training = (documents_training, np.zeros(num_documents_training))
data_test = (documents_test, np.zeros(num_documents_test))

データの生成が終わったら，実際に中身を確認してみましょう．最初のドキュメントが，25 種類の単語の生起回数で表現されているのが確認できます．25 は，上で指定した `vocabulary_size` がそのまま使われています．

In [ ]:
print('First training document = {}'.format(documents[0]))
print('\nVocabulary size = {}'.format(vocabulary_size))
print('Shape of training data = {}'.format(documents.shape))

また併せて，最初のドキュメントに含まれる各トピックの割合をみてみます．トピック数は同じく，上で指定した `num_topics` が使われています．

In [ ]:
np.set_printoptions(precision=4, suppress=True)

print('Known topic mixture of first training document = {}'.format(topic_mixtures_training[0]))
print('\nNumber of topics = {}'.format(num_topics))

また，最初の 10 ドキュメントについて，25 種類の単語がそれぞれ何回含まれているかを可視化してみると，以下のように表現できます．

In [ ]:
%matplotlib inline

fig = plot_topic_data(documents_training[:10], nrows=2, ncols=5, cmap='gray_r', with_colorbar=False)
fig.suptitle('Example Documents')
fig.set_dpi(160)

## データのロード

SageMaker の学習時につかうデータは，S3 に置く必要があります．ここでは，データを SageMaker SDK が用意しているメソッドを用いて，RecordIO フォーマットに変換します．その上で S3 にアップロードする形をとります．`write_numpy_to_dense_tensor` については[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/amazon/common.py#L88-L110)を参照ください．なお，ここではパフォーマンスが得られる RecordIO 形式に直していますが，CSV 形式のままアップロードして学習を行うことも可能です．詳細は[こちら](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/ntm.html#NTM-inputoutput)を参照ください．

ここでは自動生成データのため，ボキャブラリーと実際の単語のマッピングを行なっていませんが，実際に使用する際には，ボキャブラリー内のこの整数 ID で示される単語はこれだ，というマッピングを行いたいことがあるかと思います．その際には，補助語彙チャネルとして vocab.txt というファイルを用意して，学習時に読み込ませることで，マッピングを行うことができます．詳細については，[こちらのブログ記事](https://aws.amazon.com/jp/blogs/news/amazon-sagemaker-neural-topic-model-now-supports-auxiliary-vocabulary-channel-new-topic-evaluation-metrics-and-training-subsampling/)を参照ください．この補助語彙チャネルを用いることで，単語埋め込みトピックコヒーレンスメトリクス (WETC) やトピック一貫性メトリクス (TU) を利用することも可能となります．ざっくりまとめると，WETC は同一トピック内の単語の類似性を 0-1 で，TU はトピック内の単語が他のトピックにどれだけ現れるかを (1/K)-1 (K はトピック数)で表す指標になります．ともに高いほど，トピックがうまく分割できていることを示します．

In [ ]:
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, data_training[0].astype('float32'))
buf.seek(0)

key = 'ntm.data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)

## モデルの学習を実行

データの準備ができたら，さっそく学習ジョブを実行しましょう．ここでは，SageMaker SDK で用意されている関数を使って，ビルトインアルゴリズムのコンテナイメージ ID を取得します．

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'ntm')

Neural Topic Model では，以下の　2 つのハイパーパラメタを指定できます．

* **`num_topics`** - 推定するトピック数．ここでは，ディリクレ分布で生成した際のトピック数である 5 をそのまま使用します．

* **`feature_dim`** - ボキャブラリーの数を指定します．こちらも上で指定した単語数 25 をそのまま用います．

In [ ]:
ntm = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.c4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
ntm.set_hyperparameters(num_topics=num_topics,
                        feature_dim=vocabulary_size)

ntm.fit({'train': s3_train_data})

## モデルの推論を実行

推論を行うために，Estimateor オブジェクトからモデルオブジェクトを作成した上で，モデルをエンドポイントにデプロイします．deploy() メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．モデルのデプロイには 10 分程度時間がかかります．

In [ ]:
ntm_predictor = ntm.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

デプロイが終わったら，実際に推論を行ってみましょう．学習で使ったのと同じ形式のデータをリクエストで送ることで，5 つのトピックがそれぞれ含まれる確率を，レスポンスとして得ることができます．ここではシリアライザを csv に，でシリアライザを　json に指定しています．これによりリクエストで送られる nparray データを csv 形式に変換して推論エンドポイントに送り，またレスポンスを json に変換して受け取ることができます．推論形式の詳細については，[こちら](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/cdf-inference.html) を参照ください．

In [ ]:
ntm_predictor.content_type = 'text/csv'
ntm_predictor.serializer = csv_serializer
ntm_predictor.deserializer = json_deserializer

In [ ]:
results = ntm_predictor.predict(documents_training[:10])
print(results)

上記の結果は，以下のような形式の json データです．このままだと見辛いので，少し加工して，見やすくしてみましょう．
```
{
  'predictions': [
    {'topic_weights': [ ... ] },
    {'topic_weights': [ ... ] },
    {'topic_weights': [ ... ] },
    ...
  ]
}
```

In [ ]:
predictions = np.array([prediction['topic_weights'] for prediction in results['predictions']])

print(predictions)

同じ文章について，おおもとのトピック分布と，推定されたトピック分布を比べてみましょう．

In [ ]:
print(topic_mixtures_training[0])  # known topic mixture
print(predictions[0])  # computed topic mixture

データを 1000 行ずつ投げて推論をしてみましょう．エンドポイントのリクエストボディは 5MB が上限なので，それより小さなサイズになるように，リクエストデータをうまく分割する必要があります．

In [ ]:
def predict_batches(data, rows=1000):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = []
    for array in split_array:
        results = ntm_predictor.predict(array)
        predictions += [r['topic_weights'] for r in results['predictions']]
    return np.array(predictions)

In [ ]:
predictions = predict_batches(documents_training)

In [ ]:
predictions[0:10]

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sess.delete_endpoint(ntm_predictor.endpoint)